# **Feature engineering dos microdados do ENEM 2022**

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Carrega os dados do arquivo da etapa anterior
file_path = '../data/df_encoded.csv'
df = pd.read_csv(file_path, sep=';', encoding='iso-8859-1')
df

,TP_FAIXA_ETARIA,Q002,Q005,Q006,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO,TP_PRESENCA_LC,...,TP_ESCOLA_Pública,TP_COR_RACA_Amarela,TP_COR_RACA_Branca,TP_COR_RACA_Indígena,TP_COR_RACA_Parda,TP_COR_RACA_Preta,IN_TREINEIRO_Sim,Q022_Sim,Q024_Sim,Q025_Sim
0,12,5,3,1,NaN,NaN,NaN,NaN,NaN,Faltou,...,0,0,0,0,0,1,0,0,0,0
1,12,1,3,0,NaN,NaN,NaN,NaN,NaN,Faltou,...,0,0,1,0,0,0,0,1,1,1
2,3,2,2,1,498.8,546.0,421.1,565.3,760.0,Presente,...,0,0,0,0,0,1,0,1,0,1
3,4,5,3,16,357.8,388.6,490.7,416.0,320.0,Presente,...,0,0,0,0,1,0,0,1,1,1
4,2,6,2,1,NaN,NaN,NaN,NaN,NaN,Faltou,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,9,NaN,NaN,NaN,NaN,NaN,Faltou,...,1,0,1,0,0,0,0,1,0,1
3476104,12,5,3,1,NaN,NaN,NaN,NaN,NaN,Faltou,...,1,0,1,0,0,0,0,1,1,1
3476105,0,0,4,10,583.3,627.0,527.9,637.1,660.0,Presente,...,1,0,1,0,0,0,0,1,1,1
3476106,1,2,5,8,NaN,NaN,NaN,NaN,NaN,Faltou,...,0,0,0,0,1,0,0,1,1,1


## **Combinação de Features**

In [2]:
df_eng = df.copy()

# Calcula a média das notas objetivas e descarta as colunas individuais
df_eng['media_objetivas'] = df_eng[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT']].mean(axis=1)
df_eng['media_geral'] = df_eng[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)
df_eng.drop(columns=['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT'], inplace=True)

# Combina as informações de faltas (se faltou em qualquer prova recebe 1, caso contrário, 0)
df_eng['faltou'] = df_eng[['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT']].apply(
    lambda row: any(valor in ['Faltou', 'Eliminado'] for valor in row), axis=1
).astype(int)
df_eng.drop(columns=['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT'], inplace=True)

# Combina as colunas de raça em duas categorias: branco_amarelo e preto_pardo_indigena
df_eng['branco_amarelo'] = (df_eng['TP_COR_RACA_Branca'] + df_eng['TP_COR_RACA_Amarela']).clip(0, 1)
df_eng['preto_pardo_indigena'] = (df_eng['TP_COR_RACA_Preta'] + df_eng['TP_COR_RACA_Parda'] + df_eng['TP_COR_RACA_Indígena']).clip(0, 1)
df_eng.drop(columns=['TP_COR_RACA_Branca', 'TP_COR_RACA_Amarela', 'TP_COR_RACA_Preta', 'TP_COR_RACA_Parda', 'TP_COR_RACA_Indígena', 'TP_ESCOLA_Pública'], inplace=True)

# Junta Q022 (celular, Q024 (computador) e Q025 (internet) em uma única coluna 'acesso_tecnologia', que recebe 1 se o candidato tem qualquer um, ou 0 se não tiver nenhum
# df_eng['acesso_tecnologia'] = (df_eng['Q024_Sim'] + df_eng['Q022_Sim'] + df_eng['Q025_Sim']).clip(0, 1)
df_eng['possui_computador'] = df_eng['Q024_Sim']
df_eng.drop(columns=['Q022_Sim', 'Q024_Sim', 'Q025_Sim'], inplace=True)

# Renomeia as colunas, removendo prefixos e sufixos, convertendo para minúsculas e removendo acentos
renomear = {
    'NU_NOTA_REDACAO': 'redacao',
    'TP_ESCOLA_Privada': 'escola_privada',
    'TP_SEXO_Feminino': 'sexo_feminino',
    'IN_TREINEIRO_Sim': 'treineiro',
    'Q002': 'escolaridade_mae',
    'Q005': 'tamanho_familia',
    'Q006': 'renda_familiar',
    'TP_FAIXA_ETARIA': 'faixa_etaria'
}
df_eng.rename(columns=renomear, inplace=True)
df_eng.columns = df_eng.columns.str.lower()
df_eng

,faixa_etaria,escolaridade_mae,tamanho_familia,renda_familiar,redacao,sexo_feminino,escola_privada,treineiro,media_objetivas,media_geral,faltou,branco_amarelo,preto_pardo_indigena,possui_computador
0,12,5,3,1,NaN,0,0,0,NaN,NaN,1,0,1,0
1,12,1,3,0,NaN,0,0,0,NaN,NaN,1,1,0,1
2,3,2,2,1,760.0,1,0,0,507.800,558.24,0,0,1,0
3,4,5,3,16,320.0,0,0,0,413.275,394.62,0,0,1,1
4,2,6,2,1,NaN,0,0,0,NaN,NaN,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,9,NaN,0,0,0,NaN,NaN,1,1,0,0
3476104,12,5,3,1,NaN,1,0,0,NaN,NaN,1,1,0,1
3476105,0,0,4,10,660.0,0,0,0,593.825,607.06,0,1,0,1
3476106,1,2,5,8,NaN,0,1,0,NaN,NaN,1,0,1,1


## **Considerações sobre ponderação arbitrária**

As colunas de renda familiar, faixa etária e escolaridade da mãe representam faixas de valores, e não números contínuos. Como cada faixa cobre um intervalo, **existe um grau de ponderação arbitrária ao tratá-las como valores numéricos**. Isso ocorre porque os intervalos não possuem necessariamente a mesma amplitude, e a diferença entre faixas pode não ser linear. No entanto, essa abordagem simplifica a modelagem e evita a explosão dimensional que ocorreria com One Hot Encoding. **Optou-se por esse modelo mais simples para manter a interpretabilidade e a eficiência computacional.**

In [3]:
# Valores da coluna 'renda_familiar':
#  0 → Nenhuma renda
#  1 → Até R$ 1.212,00
#  2 → De R$ 1.212,01 até R$ 1.818,00
#  3 → De R$ 1.818,01 até R$ 2.424,00
#  4 → De R$ 2.424,01 até R$ 3.030,00
#  5 → De R$ 3.030,01 até R$ 3.636,00
#  6 → De R$ 3.636,01 até R$ 4.848,00
#  7 → De R$ 4.848,01 até R$ 6.060,00
#  8 → De R$ 6.060,01 até R$ 7.272,00
#  9 → De R$ 7.272,01 até R$ 8.484,00
# 10 → De R$ 8.484,01 até R$ 9.696,00
# 11 → De R$ 9.696,01 até R$ 10.908,00
# 12 → De R$ 10.908,01 até R$ 12.120,00
# 13 → De R$ 12.120,01 até R$ 14.544,00
# 14 → De R$ 14.544,01 até R$ 18.180,00
# 15 → De R$ 18.180,01 até R$ 24.240,00
# 16 → Mais de R$ 24.240,00

# Valores da coluna 'faixa_etaria':
#  0 → Menor de 17 anos
#  1 → 17 anos
#  2 → 18 anos
#  3 → 19 anos
#  4 → 20 anos
#  5 → 21 anos
#  6 → 22 anos
#  7 → 23 anos
#  8 → 24 anos
#  9 → 25 anos
# 10 → Entre 26 e 30 anos
# 11 → Entre 31 e 35 anos
# 12 → Entre 36 e 40 anos
# 13 → Entre 41 e 45 anos
# 14 → Entre 46 e 50 anos
# 15 → Entre 51 e 55 anos
# 16 → Entre 56 e 60 anos
# 17 → Entre 61 e 65 anos
# 18 → Entre 66 e 70 anos
# 19 → Maior de 70 anos

# Valores da coluna 'escolaridade_mae':
# 0 → Nunca estudou.
# 1 → Não completou a 4ª série/5º ano do Ensino Fundamental.
# 2 → Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
# 3 → Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
# 4 → Completou o Ensino Médio, mas não completou a Faculdade.
# 5 → Completou a Faculdade, mas não completou a Pós-graduação.
# 6 → Completou a Pós-graduação.
# 7 → Não sei - foi substituído pela moda da coluna

In [4]:
# Calcula os limiares para classificação binária, considerando a média das colunas mais um desvio padrão
media_limite = df_eng['media_objetivas'].mean() + df_eng['media_objetivas'].std()
redacao_limite = df_eng['redacao'].mean() + df_eng['redacao'].std()
media_geral_limite = df_eng['media_geral'].mean() + df_eng['media_geral'].std()
print(f'Limite para a média das objetivas ser considerada alta: {round(media_limite, 2)}')
print(f'Limite para a nota da redação ser considerada alta: {round(redacao_limite, 2)}')
print(f'Limite para a média geral ser considerada alta: {round(media_geral_limite, 2)}')

# Cria as variáveis binárias
df_eng['media_objetivas_alta'] = (df_eng['media_objetivas'] > media_limite).astype(int)
df_eng['nota_redacao_alta'] = (df_eng['redacao'] > redacao_limite).astype(int)
df_eng['media_geral_alta'] = (df_eng['media_geral'] > media_geral_limite).astype(int)

# Remove as colunas das notas
df_eng.drop(columns=['media_objetivas', 'redacao', 'media_geral'], inplace=True)
df_eng

Limite para a média das objetivas ser considerada alta: 593.73
Limite para a nota da redação ser considerada alta: 830.69
Limite para a média geral ser considerada alta: 630.96


,faixa_etaria,escolaridade_mae,tamanho_familia,renda_familiar,sexo_feminino,escola_privada,treineiro,faltou,branco_amarelo,preto_pardo_indigena,possui_computador,media_objetivas_alta,nota_redacao_alta,media_geral_alta
0,12,5,3,1,0,0,0,1,0,1,0,0,0,0
1,12,1,3,0,0,0,0,1,1,0,1,0,0,0
2,3,2,2,1,1,0,0,0,0,1,0,0,0,0
3,4,5,3,16,0,0,0,0,0,1,1,0,0,0
4,2,6,2,1,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,9,0,0,0,1,1,0,0,0,0,0
3476104,12,5,3,1,1,0,0,1,1,0,1,0,0,0
3476105,0,0,4,10,0,0,0,0,1,0,1,1,0,0
3476106,1,2,5,8,0,1,0,1,0,1,1,0,0,0


In [5]:
# Salva o dataframe para a etapa seguinte (modelagem)
df_eng.to_csv('../data/df_eng.csv', sep=';', encoding='iso-8859-1', index=False)